In [1]:
### data preparatiion 
imsz=128  # image size
epoch=50  # epoch cycle

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Init Plugin
Init Graph Optimizer
Init Kernel


In [4]:
# あらかじめ作ったディレクトリをラベル、その中の画像をデータにします。
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../../../../../GitHubData/Datascience_AI/CV/CNN/data_class',
        target_size=(imsz, imsz),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '../../../../../GitHubData/Datascience_AI/CV/CNN/data',
        target_size=(imsz, imsz),
        batch_size=24,
        class_mode='categorical')


Found 126 images belonging to 9 classes.
Found 72 images belonging to 9 classes.


In [5]:
from tensorflow.keras import datasets, layers, models
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR) 
tf.autograph.set_verbosity(0)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

target_num=9

# example of 4 layered convnet
model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(imsz, imsz, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

# classifier on top of the convnet
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(target_num, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
import time
# 処理前の時刻
t1 = time.time() 
#
res=model.fit(train_generator, steps_per_epoch=4, validation_data=validation_generator, validation_steps=2, epochs=epoch)

# 処理後の時刻
t2 = time.time()
 
import socket
# コンピュータ名を取得
host = socket.gethostname()
print(host)  

# 経過時間を表示
elapsed_time = t2-t1
print(f"処理時間：{elapsed_time}")

Num GPUs Available:  1
Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
con

2021-12-17 10:52:45.666348: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-17 10:52:45.666526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-17 10:52:48.720544: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-17 10:52:48.720661: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-17 10:52:48.848574: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50
4/4 [==============================] - ETA: 0s - loss: 2.2207 - accuracy: 0.1111

2021-12-17 10:53:01.605363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 19s 5s/step - loss: 2.2207 - accuracy: 0.1111 - val_loss: 2.1883 - val_accuracy: 0.1458
Epoch 2/50
4/4 [==============================] - 17s 5s/step - loss: 2.1965 - accuracy: 0.0873 - val_loss: 2.1905 - val_accuracy: 0.1250
Epoch 3/50
4/4 [==============================] - 17s 5s/step - loss: 2.1942 - accuracy: 0.1111 - val_loss: 2.1867 - val_accuracy: 0.0625
Epoch 4/50
4/4 [==============================] - 17s 5s/step - loss: 2.1804 - accuracy: 0.1111 - val_loss: 2.1557 - val_accuracy: 0.3125
Epoch 5/50
4/4 [==============================] - 17s 5s/step - loss: 2.1497 - accuracy: 0.1508 - val_loss: 2.1126 - val_accuracy: 0.1042
Epoch 6/50
4/4 [==============================] - 17s 5s/step - loss: 2.1086 - accuracy: 0.1746 - val_loss: 1.9828 - val_accuracy: 0.2708
Epoch 7/50
4/4 [==============================] - 17s 5s/step - loss: 1.9987 - accuracy: 0.2381 - val_loss: 1.9143 - val_accuracy: 0.2083
Epoch 8/50
4/4 [=============================

In [ ]:
# result(res)に出力されたデータをプロットする。
# 図と計算結果のモデルを保存する。

import matplotlib.pyplot as plt
%matplotlib inline

acc=res.history['accuracy']
loss=res.history['loss']
val_acc=res.history['val_accuracy']
val_loss=res.history['val_loss']
epochs=range(1,len(acc)+1)
fig, ax1= plt.subplots(figsize=(10,8))
ax2=ax1.twinx()
ax1.plot(epochs, acc, '-go', label='training accuracy')
ax2.plot(epochs, loss, '-c^', label='training loss')
ax1.plot(epochs, val_acc, '-bo', label='validation accuracy')
ax2.plot(epochs, val_loss, '-m^', label='validation loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('accuracy')
ax2.set_ylabel('loss')
ax1.legend()
ax2.legend()
plt.savefig('./history/CNN_from_directory_'+target_class+str(imsz)+'_'+str(epoch)+'epo.png')
plt.show()
# modelを保存する。
model.save('../../GitHubData/CNN/model/CNN_from_directory_'+target_class+str(imsz)+'_'+str(epoch)+'epo.h5')

# 

In [ ]:
hdf_source='../../GitHubData/CNN/model/CNN_from_directory_index128_５0epo.h5'

new_model = tf.keras.models.load_model(hdf_source)

X,Y=validation_generator.__next__()
true_classes=np.argmax(Y, axis=1)
preds_ft = new_model.predict(X)
pred_classes_ft = np.argmax(preds_ft, axis=1)

# Get the names of classes
class_names=['A','B','C','D','E','F','G','H','I']


fig, ax= plt.subplots(nrows=batch_size_is, ncols=2, figsize=(8, 40))
for i in range(batch_size_is):
    best_5=np.argsort(preds_ft[i])[-5:]
    ax[i,0].imshow(a_ar[i]/ 2 + 0.5)
    ax[i,0].set_title(str(np.array(class_names)[true_classes[i]]),fontsize=10)
    ax[i,0].axis('off')
    label=np.array(class_names)[best_5]
    ax[i,1].barh(label,preds_ft[i][best_5])
    ax[i,1].set_yticks([0, 1, 2, 3, 4])
    ax[i,1].set_yticklabels(label,rotation=0,ha='right', fontsize=10)       
fig.tight_layout()
plt.savefig('./prediction/prediction_from_directory_tmp.jpg',dpi=200)
plt.show()